In [1]:
import pandas as pd
import json
from mstrio.connection import Connection
from mstrio.api import reports,change_journal
from mstrio.project_objects.report import Report
from mstrio.utils import parser
from datetime import date, timedelta,datetime
from mstrio.project_objects.datasets  import super_cube


def open_Instance(conn, report_id):
    rep_instance = reports.report_instance(connection=conn, report_id=report_id)
    return rep_instance.json()["instanceId"]
    
def set_inst_prompt_ans(conn, report_id, instance_id, prompt_answ):
    prompt_answ_url = f'{conn.base_url}/api/reports/{report_id}/instances/{instance_id}/prompts/answers'
    ret_prompt_ans = conn.put(prompt_answ_url, data=prompt_answ)
    return ret_prompt_ans

def report_dict(conn, report_id, instance_id):
    report_ds = reports.report_instance_id(connection=conn, report_id=report_id, instance_id=instance_id)

    report_dict = parser.Parser(report_ds.json())._Parser__map_attributes(report_ds.json())
    return report_dict

def rep_to_dataframe( conn, report_id, instance_id):
    #there is a mstrio standard method for this, which I love to use
    #however, in a few cases it brings back an error
    att_col_l = []
    rep_def_resp = reports.report_definition(connection=conn, report_id=report_id)
    for col in rep_def_resp.json()["definition"]["grid"]["rows"]:

        #print(col["name"])
        for col_form in col["forms"]:
            att_col_l.append(str(col["name"]).replace(" ", "_" ) + '@' + str(col_form["name"]).replace(" ", "_" ))

    report_ds = reports.report_instance_id(connection=conn, report_id=report_id, instance_id=instance_id)

    # create attribute data frame, then re-map integer array with corresponding attribute element values  # noqa
    r=parser.Parser(report_ds.json())
    r.parse(report_ds.json())
    attribute_df = pd.DataFrame(
        data=r._mapped_attributes, columns=att_col_l
    )

    metric_df = pd.DataFrame(data=r._metric_values_raw, columns=r._metric_col_names)
    mstr_df=pd.concat([attribute_df, metric_df], axis=1)

    return mstr_df


def bld_date_prp_ans(date_l,prompt_id,date_att_id):
    prp_element_d_l=[]
    for date_str in date_l:
        parsed_date = datetime.strptime(date_str, "%m-%d-%Y")
        day = parsed_date.day
        month = parsed_date.month
        year = parsed_date.year
        prp_element_d={"id": f"h{str(month)}/{str(day)}/{str(year)};{date_att_id}"}
        prp_element_d_l.append(prp_element_d.copy())
    dates_prp_ans_d= {"id": prompt_id ,"type":"ELEMENTS", "answers":  prp_element_d_l }

    
    return dates_prp_ans_d
        
def upload_cube_mult_table(conn, mtdi_id=None, tbl_upd_dict=None,
                           cube_name=None, folder_id=None, force=False):

    if mtdi_id ==None:
        ds = super_cube.SuperCube(connection=conn, name=cube_name)
        for t in tbl_upd_dict:
            ds.add_table(name=t["tbl_name"],
                         data_frame=t["df"],
                         update_policy=t["update_policy"])
        ds.create(folder_id=folder_id,force=force)
    else:
        ds = super_cube.SuperCube(connection=conn, id=mtdi_id)
        for t in tbl_upd_dict:
            ds.add_table(name=t["tbl_name"],
                         data_frame=t["df"],
                         update_policy=t["update_policy"])
        ds.update()

    return ds.id

def get_date_str_l(start_date_str,end_date_str):
    start_date = datetime.strptime(start_date_str, "%m-%d-%Y").date()
    end_date=datetime.strptime(end_date_str, "%m-%d-%Y").date()
    # Today's date
    today = datetime.today().date()
    
    # Create a list of all dates from start to today
    date_list = pd.date_range(start=start_date, end=end_date).to_list()
    
    # If you want them as strings:
    date_str_list = [d.strftime("%m-%d-%Y") for d in date_list]
    return date_str_list

def lu_object_type(pa_obj_type_id):
    # I didn't found the obeject type_id you need to work using the REST API
    # thus I'm using this method to work around this problem
    pa_obj_d_l= [ {'PA_OBJ_TYPE_ID': '6', 'PA_OBJ_TYPE_DESC': 'Custom Group', 'MD_OBJ_TYPE_ID': '1',
         'MD_OBJ_TYPE_BEZ': 'Filter'},
        {'PA_OBJ_TYPE_ID': '8', 'PA_OBJ_TYPE_DESC': 'Data Import Cube', 'MD_OBJ_TYPE_ID': '3',
         'MD_OBJ_TYPE_BEZ': 'Report'},
        {'PA_OBJ_TYPE_ID': '16', 'PA_OBJ_TYPE_DESC': 'Filter', 'MD_OBJ_TYPE_ID': '1', 'MD_OBJ_TYPE_BEZ': 'Filter'},
        {'PA_OBJ_TYPE_ID': '40', 'PA_OBJ_TYPE_DESC': 'Template', 'MD_OBJ_TYPE_ID': '2',
         'MD_OBJ_TYPE_BEZ': 'Template'},
        {'PA_OBJ_TYPE_ID': '31', 'PA_OBJ_TYPE_DESC': 'SQL Report', 'MD_OBJ_TYPE_ID': '3',
         'MD_OBJ_TYPE_BEZ': 'Report'},
        {'PA_OBJ_TYPE_ID': '19', 'PA_OBJ_TYPE_DESC': 'SQL Report', 'MD_OBJ_TYPE_ID': '3',
         'MD_OBJ_TYPE_BEZ': 'Report'},
        {'PA_OBJ_TYPE_ID': '23', 'PA_OBJ_TYPE_DESC': 'Metric', 'MD_OBJ_TYPE_ID': '4', 'MD_OBJ_TYPE_BEZ': 'Metric'},
        {'PA_OBJ_TYPE_ID': '107', 'PA_OBJ_TYPE_DESC': 'User Folder', 'MD_OBJ_TYPE_ID': '8',
         'MD_OBJ_TYPE_BEZ': 'Folder'},
        {'PA_OBJ_TYPE_ID': '2', 'PA_OBJ_TYPE_DESC': 'Attribute Element Prompt', 'MD_OBJ_TYPE_ID': '10',
         'MD_OBJ_TYPE_BEZ': 'Prompt'},
        {'PA_OBJ_TYPE_ID': '37', 'PA_OBJ_TYPE_DESC': 'Embedded Prompt', 'MD_OBJ_TYPE_ID': '10',
         'MD_OBJ_TYPE_BEZ': 'Prompt'},
        {'PA_OBJ_TYPE_ID': '58', 'PA_OBJ_TYPE_DESC': 'Level prompt', 'MD_OBJ_TYPE_ID': '10',
         'MD_OBJ_TYPE_BEZ': 'Prompt'},
        {'PA_OBJ_TYPE_ID': '24', 'PA_OBJ_TYPE_DESC': 'Object Prompt', 'MD_OBJ_TYPE_ID': '10',
         'MD_OBJ_TYPE_BEZ': 'Prompt'},
        {'PA_OBJ_TYPE_ID': '28', 'PA_OBJ_TYPE_DESC': 'Prompt', 'MD_OBJ_TYPE_ID': '10', 'MD_OBJ_TYPE_BEZ': 'Prompt'},
        {'PA_OBJ_TYPE_ID': '57', 'PA_OBJ_TYPE_DESC': 'Prompt Expression Draft', 'MD_OBJ_TYPE_ID': '10',
         'MD_OBJ_TYPE_BEZ': 'Prompt'},
        {'PA_OBJ_TYPE_ID': '36', 'PA_OBJ_TYPE_DESC': 'Value Prompt', 'MD_OBJ_TYPE_ID': '10',
         'MD_OBJ_TYPE_BEZ': 'Prompt'},
        {'PA_OBJ_TYPE_ID': '1', 'PA_OBJ_TYPE_DESC': 'Attribute', 'MD_OBJ_TYPE_ID': '12',
         'MD_OBJ_TYPE_BEZ': 'Attribute'},
        {'PA_OBJ_TYPE_ID': '15', 'PA_OBJ_TYPE_DESC': 'Fact', 'MD_OBJ_TYPE_ID': '13', 'MD_OBJ_TYPE_BEZ': 'Fact'},
        {'PA_OBJ_TYPE_ID': '102', 'PA_OBJ_TYPE_DESC': 'System Hierarchy', 'MD_OBJ_TYPE_ID': '14',
         'MD_OBJ_TYPE_BEZ': 'Dimension'},
        {'PA_OBJ_TYPE_ID': '103', 'PA_OBJ_TYPE_DESC': 'User Hierarchy', 'MD_OBJ_TYPE_ID': '14',
         'MD_OBJ_TYPE_BEZ': 'Dimension'},
        {'PA_OBJ_TYPE_ID': '35', 'PA_OBJ_TYPE_DESC': 'Transformation', 'MD_OBJ_TYPE_ID': '43',
         'MD_OBJ_TYPE_BEZ': 'Role'},
        {'PA_OBJ_TYPE_ID': '5', 'PA_OBJ_TYPE_DESC': 'Consolidation', 'MD_OBJ_TYPE_ID': '47',
         'MD_OBJ_TYPE_BEZ': 'Consolidation'},
        {'PA_OBJ_TYPE_ID': '7', 'PA_OBJ_TYPE_DESC': 'Document', 'MD_OBJ_TYPE_ID': '55',
         'MD_OBJ_TYPE_BEZ': 'DocumentDefinition'},
        {'PA_OBJ_TYPE_ID': '14', 'PA_OBJ_TYPE_DESC': 'Dossier', 'MD_OBJ_TYPE_ID': '55',
         'MD_OBJ_TYPE_BEZ': 'DocumentDefinition'},
        {'PA_OBJ_TYPE_ID': '110', 'PA_OBJ_TYPE_DESC': 'Security Filter', 'MD_OBJ_TYPE_ID': '58',
         'MD_OBJ_TYPE_BEZ': 'Security Filter'},
        {'PA_OBJ_TYPE_ID': '-1', 'PA_OBJ_TYPE_DESC': 'Search', 'MD_OBJ_TYPE_ID': '39', 'MD_OBJ_TYPE_BEZ': 'Search'},
        {'PA_OBJ_TYPE_ID': '-2', 'PA_OBJ_TYPE_DESC': 'Shortcut', 'MD_OBJ_TYPE_ID': '18',
         'MD_OBJ_TYPE_BEZ': 'Shortcut'}]
    for obj_d in pa_obj_d_l:
        if pa_obj_type_id ==obj_d["PA_OBJ_TYPE_ID"]:
            return obj_d["MD_OBJ_TYPE_ID"]

def run_pa_report(pa_conn,pa_report_id,Elements_of_Project_prp_id,Project_att_element_id,Project_attribute_id,
                  Elements_of_date_prp_id,Date_attribute_id,start_date_str,end_date_str):
    
    date_str_list=get_date_str_l(start_date_str,end_date_str)
    project_prp_ans_d={"id":Elements_of_Project_prp_id ,"type":"ELEMENTS", "answers": [{'id': f'h{Project_att_element_id};{Project_attribute_id}'}]}
    dates_prp_ans_d=bld_date_prp_ans(date_l=date_str_list,prompt_id=Elements_of_date_prp_id ,date_att_id=Date_attribute_id) 
    
    prompt_answ_j=json.dumps({"prompts": [project_prp_ans_d,dates_prp_ans_d]})
    prompt_answ_j1=prompt_answ_j
    instance_id= open_Instance(conn=pa_conn, report_id=pa_report_id)
    p_ans_resp=set_inst_prompt_ans(conn=pa_conn, report_id=pa_report_id,instance_id=instance_id, prompt_answ=prompt_answ_j)
    
    
    unfilt_df=rep_to_dataframe(conn=pa_conn, report_id=report_id,instance_id=instance_id)
    return unfilt_df

def prepare_cube_df(unfilt_df):
    remove_deleted_obj_l = unfilt_df.loc[unfilt_df['Change_Type@Name'] == "Delete Objects", 'Object@GUID'].unique()
    
    # Filter out all rows with these obj_ids
    exist_obj_df = unfilt_df[~unfilt_df['Object@GUID'].isin(remove_deleted_obj_l)]
    change_name_d={'Object@GUID': 'object_id','Object@Name': 'object_name'}
    pa_change_log_df= exist_obj_df.rename(columns=change_name_d)
    pa_change_log_df["object_type_val"] = pa_change_log_df['Object_Type@ID'].apply(lambda x: lu_object_type(x))
    print(exist_obj_df.columns)
    pa_change_log_df['mod_date'] =pd.to_datetime(pa_change_log_df["Timestamp@UTC"]).dt.date
    pa_change_log_df['mod_week'] =pd.to_datetime(pa_change_log_df["Timestamp@UTC"]).dt.isocalendar().week
    pa_change_log_df['mod_month'] =pd.to_datetime(pa_change_log_df["Timestamp@UTC"]).dt.month
    pa_change_log_df['mod_quarter'] =pd.to_datetime(pa_change_log_df["Timestamp@UTC"]).dt.quarter
    pa_change_log_df['mod_year'] =pd.to_datetime(pa_change_log_df["Timestamp@UTC"]).dt.year
    
    pa_change_obj_log_d_l=pa_change_log_df.to_dict(orient="records")
    pa_change_log_df=pd.DataFrame(pa_change_obj_log_d_l)
    pa_change_log_df=pa_change_log_df.astype('str')
    return pa_change_log_df

In [2]:
user_path="..\\config\\user_d.json"
with open(user_path, 'r') as file:
    user_d = json.load(file)


project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"


conn_params =  user_d["conn_params"]
conn_params["project_id"]=project_id
conn = Connection(**conn_params)
conn.headers['Content-type'] = "application/json"

cube_id="A320AC9749095F4B096D31B1C258A4A4"

cube_name="pa_change_obj_log"
cube_folder_id="CF2049E94A0000A16532D39C6D783F1D"

start_date_str='04-24-2025'
end_date_str=datetime.today().date().strftime("%m-%d-%Y") 
#end_date_str='04-26-2025'
Project_att_element_id="7224027087873511424" #pk of the project in PA


Elements_of_date_prp_id="8344B89E455FC0C6B81B7896DA0678F1"
Date_attribute_id="3286D44B4572B4CAD29A80B0E88E789C"
Elements_of_Project_prp_id="EF66D49D40D0E799E1D1909A65085E97"
Project_attribute_id="3A0BE6C741DE7CDD4A4C01925B5A04E9"

pa_conn_params={}
pa_conn_params["username"]="Administrator"
pa_conn_params["password"]="Victoria_01"
pa_conn_params["base_url"]="http://85.214.60.83:8080/MicroStrategyLibrary/api"
pa_conn_params["project_id"]="40DA7A1549651FD60A9D39AAB7EC77B0"
pa_conn = Connection(**pa_conn_params)

pa_conn.headers['Content-type'] = "application/json"
pa_report_id="CC7C431F49B2493733BCAB813479D61D"

unfilt_df=run_pa_report(pa_conn,pa_report_id,Elements_of_Project_prp_id,Project_att_element_id,Project_attribute_id,
                  Elements_of_date_prp_id,Date_attribute_id,start_date_str,end_date_str)

pa_change_log_df=prepare_cube_df(unfilt_df)


pa_change_obj_log_d_l=[{"df":pa_change_log_df,"tbl_name":"change_obj_log_df", "update_policy":"Replace" }]

cube_id=upload_cube_mult_table(conn=conn, mtdi_id=cube_id, tbl_upd_dict=pa_change_obj_log_d_l,
                          cube_name=cube_name, folder_id=cube_folder_id)
pa_change_logs_df

Connection to MicroStrategy Intelligence Server has been established.
Connection to MicroStrategy Intelligence Server has been established.


NameError: name 'report_id' is not defined

In [3]:
conn.headers

{'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'X-MSTR-AuthToken': 'sn6bm8lvpu63k0du020mq79kuq', 'X-MSTR-ProjectID': 'B7CA92F04B9FAE8D941C3E9B7E0CD754', 'Content-type': 'application/json'}

In [4]:
dir(conn)

['_Connection__check_version',
 '_Connection__configure_session',
 '_Connection__get_user_info',
 '_Connection__password',
 '_Connection__prompt_credentials',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_configure_ssl',
 '_get_authorization',
 '_get_session_timeout',
 '_is_session_expired',
 '_iserver_version',
 '_login',
 '_renew',
 '_renew_or_reconnect',
 '_request',
 '_reset_timeout',
 '_session',
 '_status',
 '_user_full_name',
 '_user_id',
 '_user_initials',
 '_validate_project_selected',
 '_web_version',
 'base_url',
 'certificate_path',
 'close',
 'connect',
 'delegate',
 'delete',
 'environment',
 'get',
 'get_id

In [10]:
conn.headers

{'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'X-MSTR-AuthToken': 'sn6bm8lvpu63k0du020mq79kuq', 'X-MSTR-ProjectID': 'B7CA92F04B9FAE8D941C3E9B7E0CD754', 'Content-type': 'application/json'}

In [15]:
conn.close()
conn.status()

Connection to MicroStrategy Intelligence Server has been closed.
Connection to MicroStrategy Intelligence Server is not active.


False

In [24]:
if conn.status()==0:
    print("Hallo")
else:
    print("EEE")
conn.status()

Connection to MicroStrategy Intelligence Server is not active.
Hallo
Connection to MicroStrategy Intelligence Server is not active.


False

In [12]:
dir(conn)

['_Connection__check_version',
 '_Connection__configure_session',
 '_Connection__get_user_info',
 '_Connection__password',
 '_Connection__prompt_credentials',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_configure_ssl',
 '_get_authorization',
 '_get_session_timeout',
 '_is_session_expired',
 '_iserver_version',
 '_login',
 '_renew',
 '_renew_or_reconnect',
 '_request',
 '_reset_timeout',
 '_session',
 '_status',
 '_user_full_name',
 '_user_id',
 '_user_initials',
 '_validate_project_selected',
 '_web_version',
 'base_url',
 'certificate_path',
 'close',
 'connect',
 'delegate',
 'delete',
 'environment',
 'get',
 'get_id

In [21]:
conn.status()

Connection to MicroStrategy Intelligence Server is not active.


False